### Creating config files

In [15]:
import configparser
config = configparser.ConfigParser()

config.add_section('Constants')
config.set('Constants', 'batch_size','32')
config.set('Constants', 'num_classes','10')
config.set('Constants', 'epochs','100')
config.set('Constants', 'data_augmentation','True')
config.set('Constants', 'num_predictions','20')

with open('/Spring 2018/AI/Assignment-1/configuration-base.ini', 'w') as configfile:
    config.write(configfile)

In [9]:
import configparser
config = configparser.ConfigParser()

config.add_section('Constants')
config.set('Constants', 'batch_size','64')
config.set('Constants', 'num_classes','10')
config.set('Constants', 'epochs','100')
config.set('Constants', 'data_augmentation','True')
config.set('Constants', 'num_predictions','10')

with open('/Spring 2018/AI/Assignment-1/configuration-1.ini', 'w') as configfile:
    config.write(configfile)
    

In [10]:
import configparser
config = configparser.ConfigParser()

config.add_section('Constants')
config.set('Constants', 'batch_size','128')
config.set('Constants', 'num_classes','10')
config.set('Constants', 'epochs','200')
config.set('Constants', 'data_augmentation','True')
config.set('Constants', 'num_predictions','20')

with open('/Spring 2018/AI/Assignment-1/configuration-2.ini', 'w') as configfile:
    config.write(configfile)

In [7]:
import configparser
config = configparser.ConfigParser()

config.add_section('Constants')
config.set('Constants', 'batch_size','256')
config.set('Constants', 'num_classes','10')
config.set('Constants', 'epochs','100')
config.set('Constants', 'data_augmentation','False')
config.set('Constants', 'num_predictions','20')

with open('/Spring 2018/AI/Assignment-1/configuration-3.ini', 'w') as configfile:
    config.write(configfile)

### Reading base parameters from config file

Reading parameters from base file test accuracy is 78%. Shows that it is a good model.

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import configparser
import os

config = configparser.ConfigParser()
config.read("/Spring 2018/AI/Assignment-1/configuration-base.ini")

batch_size = int(config.get('Constants','batch_size'))
num_classes = int(config.get('Constants','num_classes'))
epochs = int(config.get('Constants','epochs'))
data_augmentation = config.get('Constants','data_augmentation')
num_predictions = int(config.get('Constants','num_predictions'))
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 42s 27ms/step - loss: 1.8493 - acc: 0.3177 - val_loss: 1.5472 - val_acc: 0.4360
Epoch 2/100
1563/1563 [==============================] - 32s 20ms/step - loss: 1.5634 - acc: 0.4270 - val_loss: 1.4085 - val_acc: 0.4996
Epoch 3/100
1563/1563 [==============================] - 29s 19ms/step - loss: 1.4330 - acc: 0.4839 - val_loss: 1.2489 - val_acc: 0.5619
Epoch 4/100
1563/1563 [==============================] - 32s 21ms/step - loss: 1.3409 - acc: 0.5191 - val_loss: 1.1665 - val_acc: 0.5859
Epoch 5/100
1563/1563 [==============================] - 33s 21ms/step - loss: 1.2721 - acc: 0.5455 - val_loss: 1.1238 - val_acc: 0.6033
Epoch 6/100
1563/1563 [==============================] - 32s 21ms/step - loss: 1.2103 - acc: 0.5690 - val_loss: 1.0526 - val_acc: 0.6264
Epoch 7/100
1563/1563 [==============================] - 27s 17ms/step

1563/1563 [==============================] - 21s 13ms/step - loss: 0.7405 - acc: 0.7518 - val_loss: 0.6789 - val_acc: 0.7796
Epoch 60/100
1563/1563 [==============================] - 21s 14ms/step - loss: 0.7353 - acc: 0.7531 - val_loss: 0.6373 - val_acc: 0.7825
Epoch 61/100
1563/1563 [==============================] - 21s 13ms/step - loss: 0.7367 - acc: 0.7535 - val_loss: 0.6238 - val_acc: 0.7945
Epoch 62/100
1563/1563 [==============================] - 21s 13ms/step - loss: 0.7404 - acc: 0.7516 - val_loss: 0.6168 - val_acc: 0.7949
Epoch 63/100
1563/1563 [==============================] - 21s 13ms/step - loss: 0.7391 - acc: 0.7522 - val_loss: 0.6446 - val_acc: 0.7846
Epoch 64/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7406 - acc: 0.7527 - val_loss: 0.6488 - val_acc: 0.7968
Epoch 65/100
1563/1563 [==============================] - 23s 15ms/step - loss: 0.7354 - acc: 0.7540 - val_loss: 0.6188 - val_acc: 0.7953
Epoch 66/100
1563/1563 [=======================

### Reading from configuration-1.ini

batch_size = 64	
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 10

Accuracy is even better with this set of parameters 81%. There is no major difference in train and test accuracies which shows that model has learnt well.

In [11]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import configparser
import os

config = configparser.ConfigParser()
config.read("/Spring 2018/AI/Assignment-1/configuration-1.ini")

batch_size = int(config.get('Constants','batch_size'))
num_classes = int(config.get('Constants','num_classes'))
epochs = int(config.get('Constants','epochs'))
data_augmentation = config.get('Constants','data_augmentation')
num_predictions = int(config.get('Constants','num_predictions'))
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
782/782 [==============================] - 14s 18ms/step - loss: 1.9153 - acc: 0.2945 - val_loss: 1.6334 - val_acc: 0.4108
Epoch 2/100
782/782 [==============================] - 13s 16ms/step - loss: 1.6446 - acc: 0.3955 - val_loss: 1.4634 - val_acc: 0.4616
Epoch 3/100
782/782 [==============================] - 13s 17ms/step - loss: 1.5300 - acc: 0.4406 - val_loss: 1.4542 - val_acc: 0.4711
Epoch 4/100
782/782 [==============================] - 13s 17ms/step - loss: 1.4587 - acc: 0.4692 - val_loss: 1.3386 - val_acc: 0.5140
Epoch 5/100
782/782 [==============================] - 13s 17ms/step - loss: 1.3940 - acc: 0.4987 - val_loss: 1.2953 - val_acc: 0.5392
Epoch 6/100
782/782 [==============================] - 13s 16ms/step - loss: 1.3442 - acc: 0.5182 - val_loss: 1.1876 - val_acc: 0.5848
Epoch 7/100
782/782 [==============================] - 13s 16ms/step - loss: 1.305

### Reading from configuration-2.ini

batch_size = 128	
num_classes = 10
epochs = 200
data_augmentation = True
num_predictions = 20

Test accuracy is better with this set of parameters, 84% and the model has learnt well.

In [12]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import configparser
import os

config = configparser.ConfigParser()
config.read("/Spring 2018/AI/Assignment-1/configuration-2.ini")

batch_size = int(config.get('Constants','batch_size'))
num_classes = int(config.get('Constants','num_classes'))
epochs = int(config.get('Constants','epochs'))
data_augmentation = config.get('Constants','data_augmentation')
num_predictions = int(config.get('Constants','num_predictions'))
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/200
391/391 [==============================] - 12s 31ms/step - loss: 1.9790 - acc: 0.2727 - val_loss: 1.7192 - val_acc: 0.3868
Epoch 2/200
391/391 [==============================] - 11s 29ms/step - loss: 1.7259 - acc: 0.3704 - val_loss: 1.5607 - val_acc: 0.4417
Epoch 3/200
391/391 [==============================] - 11s 28ms/step - loss: 1.6131 - acc: 0.4088 - val_loss: 1.4507 - val_acc: 0.4776
Epoch 4/200
391/391 [==============================] - 11s 28ms/step - loss: 1.5346 - acc: 0.4414 - val_loss: 1.3922 - val_acc: 0.5077
Epoch 5/200
391/391 [==============================] - 11s 27ms/step - loss: 1.4765 - acc: 0.4642 - val_loss: 1.3174 - val_acc: 0.5331
Epoch 6/200
391/391 [==============================] - 11s 27ms/step - loss: 1.4249 - acc: 0.4844 - val_loss: 1.2710 - val_acc: 0.5526
Epoch 7/200
391/391 [==============================] - 11s 27ms/step - loss: 1.387

391/391 [==============================] - 12s 30ms/step - loss: 0.6300 - acc: 0.7859 - val_loss: 0.5301 - val_acc: 0.8214
Epoch 121/200
391/391 [==============================] - 12s 31ms/step - loss: 0.6285 - acc: 0.7864 - val_loss: 0.5353 - val_acc: 0.8168
Epoch 122/200
391/391 [==============================] - 12s 31ms/step - loss: 0.6223 - acc: 0.7886 - val_loss: 0.5376 - val_acc: 0.8157
Epoch 123/200
391/391 [==============================] - 12s 30ms/step - loss: 0.6215 - acc: 0.7902 - val_loss: 0.5266 - val_acc: 0.8213
Epoch 124/200
391/391 [==============================] - 12s 30ms/step - loss: 0.6243 - acc: 0.7880 - val_loss: 0.5236 - val_acc: 0.8228
Epoch 125/200
391/391 [==============================] - 12s 30ms/step - loss: 0.6218 - acc: 0.7882 - val_loss: 0.5375 - val_acc: 0.8205
Epoch 126/200
391/391 [==============================] - 11s 29ms/step - loss: 0.6218 - acc: 0.7895 - val_loss: 0.5270 - val_acc: 0.8192
Epoch 127/200
391/391 [==============================] 

### Reading from configuration-3.ini

batch_size = 256	
num_classes = 10
epochs = 100
data_augmentation = False
num_predictions = 20

The model gives a lower accuracy of 79%.

In [8]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import configparser
import os

config = configparser.ConfigParser()
config.read("/Spring 2018/AI/Assignment-1/configuration-3.ini")

batch_size = int(config.get('Constants','batch_size'))
num_classes = int(config.get('Constants','num_classes'))
epochs = int(config.get('Constants','epochs'))
data_augmentation = config.get('Constants','data_augmentation')
num_predictions = int(config.get('Constants','num_predictions'))
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/100
196/196 [==============================] - 26s 134ms/step - loss: 2.0754 - acc: 0.2298 - val_loss: 1.8548 - val_acc: 0.3462
Epoch 2/100
196/196 [==============================] - 26s 133ms/step - loss: 1.8259 - acc: 0.3360 - val_loss: 1.6977 - val_acc: 0.3986
Epoch 3/100
196/196 [==============================] - 31s 160ms/step - loss: 1.7248 - acc: 0.3717 - val_loss: 1.5891 - val_acc: 0.4334
Epoch 4/100
196/196 [==============================] - 29s 150ms/step - loss: 1.6551 - acc: 0.3976 - val_loss: 1.5595 - val_acc: 0.4378
Epoch 5/100
196/196 [==============================] - 27s 140ms/step - loss: 1.5992 - acc: 0.4166 - val_loss: 1.4726 - val_acc: 0.4695
Epoch 6/100
196/196 [==============================] - 23s 120ms/step - loss: 1.5491 - acc: 0.4344 - val_loss: 1.4501 - val_acc: 0.4862
Epoch 7/100
196/196 [==============================] - 24s 121ms/step - loss

By far, the best model is the one with configuration-2.ini.